Importing libraries

In [66]:
import pandas as pd
import numpy as np
import lxml
from geopy.geocoders import Nominatim
import json
import requests
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium

Reading the html table using pandas

In [2]:
d=pd.read_html("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")

Reading the table into a dataframe

In [22]:
df=d[0]
df.head()

,Postal code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


Dropping the rows which have "Borough" as "Not assigned"

In [7]:
df_mod1=df[df["Borough"]!="Not assigned"]
df_mod1.head()

,Postal code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
5,M6A,North York,Lawrence Manor / Lawrence Heights
6,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


Merging the rows on the basis of "Postal code" to join the "Neighborhood" values

In [17]:
df_mod1=df_mod1.groupby("Postal code").agg({"Borough":"first","Neighborhood":", ".join}).reset_index()
df_mod1.head()

,Postal code,Borough,Neighborhood
0,M1B,Scarborough,Malvern / Rouge
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek
2,M1E,Scarborough,Guildwood / Morningside / West Hill
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


Checking if we have any "Neighborhood" values as NaN

In [15]:
df_mod1["Neighborhood"].isna()

0      False
1      False
2      False
3      False
4      False
       ...  
98     False
99     False
100    False
101    False
102    False
Name: Neighborhood, Length: 103, dtype: bool

Printing the number of rows and column in the final dataframe

In [21]:
df_mod1.shape

(103, 3)

Importing the dataframe for the latitude and longitude of the neighborhoods in Toronto

In [23]:
dff=pd.read_csv("Geospatial_Coordinates.csv")
dff.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [29]:
dff.shape

(103, 3)

In [42]:
df_mod1.rename(columns={"Postal code":"postal code"},inplace=True)
dff.rename(columns={"Postal code":"postal code"},inplace=True)

In [43]:
df_mod1.head()

,postal code,Borough,Neighborhood
0,M1B,Scarborough,Malvern / Rouge
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek
2,M1E,Scarborough,Guildwood / Morningside / West Hill
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [44]:
dff.head()

,postal code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [51]:
df_modtest=df_mod1

In [52]:
df_modtest.head()

,postal code,Borough,Neighborhood
0,M1B,Scarborough,Malvern / Rouge
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek
2,M1E,Scarborough,Guildwood / Morningside / West Hill
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [53]:
df_modtest["Latitude"]=dff["Latitude"]

In [54]:
df_modtest.head()

,postal code,Borough,Neighborhood,Latitude
0,M1B,Scarborough,Malvern / Rouge,43.806686
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek,43.784535
2,M1E,Scarborough,Guildwood / Morningside / West Hill,43.763573
3,M1G,Scarborough,Woburn,43.770992
4,M1H,Scarborough,Cedarbrae,43.773136


In [59]:
df_modtest["Longitude"]=dff["Longitude"]
df_modtest.head()

,postal code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,Malvern / Rouge,43.806686,-79.194353
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek,43.784535,-79.160497
2,M1E,Scarborough,Guildwood / Morningside / West Hill,43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


Use geopy library to get the latitude and longitude values of Toronto.

In [115]:
address="Toronto"
geolocator=Nominatim(user_agent="tor_explorer")
location=geolocator.geocode(address)
latitude=location.latitude
longitude=location.longitude
print("The geographical coordinate of Toronto are {} {}.".format(latitude,longitude))

The geographical coordinate of Toronto are 43.6534817 -79.3839347.


Visualization of Toronto neighborhood on map

In [116]:
map_toronto=folium.Map(location=[latitude,longitude],zoom_start=10)
for lat,lng,borough,neighborhood in zip(df_modtest["Latitude"],df_modtest["Longitude"],df_modtest["Borough"],df_modtest["Neighborhood"]):
    label="{},{}".format(neighborhood,borough)
    label=folium.Popup(label,parse_html=True)
    folium.CircleMarker([lat,lng],radius=5,popup=label,color="blue",fill=True,fill_color="#3186cc",fill_opacity=0.7,parse_html=False).add_to(map_toronto)
map_toronto

Define Foursquare Credentials and Version

In [117]:
CLIENT_ID="DCPJ0FU31FHFYZEUHA1MNYCKEDOZUCXDFWZY4OLMGOWHQCKD"
CLIENT_SECRET="YARGMSDOC5YXEKKK0OHHSYH23UBCFMINIF3SLOZZJQ2NDFBE"
VERSION="20180605"
print("Your credentials:")
print("CLIENT_ID: "+CLIENT_ID)
print("CLIENT_SECRET: "+CLIENT_SECRET)

Your credentials:
CLIENT_ID: DCPJ0FU31FHFYZEUHA1MNYCKEDOZUCXDFWZY4OLMGOWHQCKD
CLIENT_SECRET: YARGMSDOC5YXEKKK0OHHSYH23UBCFMINIF3SLOZZJQ2NDFBE


Exploring neighborhoods in Toronto: Creating a function to get nearby venues to all neighborhoods in Toronto

In [118]:
def getNearbyVenues(names,latitudes,longitudes,radius=500,LIMIT=100):
    venues_list=[]
    for name,lat,lng in zip(names,latitudes,longitudes):
        print(name)
        url="https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(CLIENT_ID,CLIENT_SECRET,VERSION,lat,lng,radius,LIMIT)
        results=requests.get(url).json()["response"]["groups"][0]["items"]
        venues_list.append([(name,lat,lng,v["venue"]["name"],v["venue"]["location"]["lat"],v["venue"]["location"]["lng"],v["venue"]["categories"][0]["name"]) for v in results])
    nearby_venues=pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns=["Neighborhood","Neighborhood Latitude","Neighborhood Longitude","Venue","Venue Latitude","Venue Longitude","Venue Category"]
    return (nearby_venues)

Filtering the Boroughs that contain the word "Toronto"

In [109]:
dfnew1=pd.DataFrame(columns=["Postal Code","Borough","Neighborhood","Latitude","Longitude"])
for i in df_modtest.index:
    if ("Toronto" in str(df_modtest["Borough"][i])):
        #print(str(i))
        dfnew1=dfnew1.append({"Postal Code":str(df_modtest["postal code"][i]),"Borough":str(df_modtest["Borough"][i]),"Neighborhood":str(df_modtest["Neighborhood"][i]),"Latitude":str(df_modtest["Latitude"][i]),"Longitude":str(df_modtest["Longitude"][i])},ignore_index=True)

In [121]:
dfnew1.head(2)

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.67635739999999,-79.2930312
1,M4K,East Toronto,The Danforth West / Riverdale,43.6795571,-79.352188


Now running the "getNearbyVenues" function on the neighbourhoods of Toronto

In [119]:
toronto_venues=getNearbyVenues(names=dfnew1["Neighborhood"],latitudes=dfnew1["Latitude"],longitudes=dfnew1["Longitude"])

The Beaches
The Danforth West / Riverdale
India Bazaar / The Beaches West
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park / Summerhill East
Summerhill West / Rathnelly / South Hill / Forest Hill SE / Deer Park
Rosedale
St. James Town / Cabbagetown
Church and Wellesley
Regent Park / Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Richmond / Adelaide / King
Harbourfront East / Union Station / Toronto Islands
Toronto Dominion Centre / Design Exchange
Commerce Court / Victoria Hotel
Roselawn
Forest Hill North & West
The Annex / North Midtown / Yorkville
University of Toronto / Harbord
Kensington Market / Chinatown / Grange Park
CN Tower / King and Spadina / Railway Lands / Harbourfront West / Bathurst Quay / South Niagara / Island airport
Stn A PO Boxes
First Canadian Place / Underground city
Christie
Dufferin / Dovercourt Village
Little Portugal / Trinity
Brockton / Parkdale Village / Exhibition Place
High Park / 

Checking the size of the resulting dataframe

In [122]:
print(toronto_venues.shape)
toronto_venues.head()

(1624, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.67635739999999,-79.2930312,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.67635739999999,-79.2930312,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.67635739999999,-79.2930312,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beaches,43.67635739999999,-79.2930312,Upper Beaches,43.680563,-79.292869,Neighborhood
4,The Beaches,43.67635739999999,-79.2930312,Dip 'n Sip,43.678897,-79.297745,Coffee Shop


Let's check how many venues were returned for each neighborhood

In [123]:
toronto_venues.groupby("Neighborhood").count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,57,57,57,57,57,57
Brockton / Parkdale Village / Exhibition Place,23,23,23,23,23,23
Business reply mail Processing CentrE,17,17,17,17,17,17
CN Tower / King and Spadina / Railway Lands / Harbourfront West / Bathurst Quay / South Niagara / Island airport,17,17,17,17,17,17
Central Bay Street,65,65,65,65,65,65
Christie,18,18,18,18,18,18
Church and Wellesley,72,72,72,72,72,72
Commerce Court / Victoria Hotel,100,100,100,100,100,100
Davisville,37,37,37,37,37,37


Let's find out how many unique categories can be curated from all the returned venues

In [124]:
print("There are {} unique categories.".format(len(toronto_venues["Venue Category"].unique())))

There are 231 unique categories.


Analyze each neighborhood : One hot encoding

In [132]:
toronto_onehot11=pd.get_dummies(toronto_venues[["Venue Category"]],prefix="",prefix_sep="")
toronto_onehot11["Neighborhood"]=toronto_venues["Neighborhood"]
toronto_onehot11.head()

,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,...,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Bringing the "Neighborhood" column to the front

In [133]:
n=toronto_onehot11["Neighborhood"]
toronto_onehot11.drop(labels=["Neighborhood"],axis=1,inplace=True)
toronto_onehot11.insert(0,"Neighborhood",n)
toronto_onehot11.head()

,Neighborhood,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio
0,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Seeing the dimensions of the new dataframe

In [134]:
toronto_onehot11.shape

(1624, 231)

Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [147]:
toronto_grouped=toronto_onehot11.groupby("Neighborhood").mean().reset_index()
toronto_grouped

,Neighborhood,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio
0,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.000000,0.00,0.017544,0.000000,0.000000,0.000000,0.000000,0.000000
1,Brockton / Parkdale Village / Exhibition Place,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.043478
2,Business reply mail Processing CentrE,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.058824
3,CN Tower / King and Spadina / Railway Lands / ...,0.058824,0.058824,0.058824,0.117647,0.176471,0.117647,0.000000,0.000000,0.00,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,Central Bay Street,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.000000,0.00,0.015385,0.000000,0.000000,0.000000,0.000000,0.015385
5,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,Church and Wellesley,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.013889,0.000000,0.00,...,0.013889,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.027778
7,Commerce Court / Victoria Hotel,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.040000,0.000000,0.00,...,0.000000,0.000000,0.000000,0.00,0.020000,0.000000,0.000000,0.010000,0.000000,0.000000
8,Davisville,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.027027,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,Davisville North,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


Let's confirm the new shape

In [136]:
toronto_grouped.shape

(39, 231)

Let's print each neighborhood along with the top 5 most common venues

In [137]:
num_top_venues=5
for hood in toronto_grouped["Neighborhood"]:
    print("----"+hood+"----")
    temp=toronto_grouped[toronto_grouped["Neighborhood"]==hood].T.reset_index()
    temp.columns=["venue","freq"]
    temp=temp.iloc[1:]
    temp["freq"]=temp["freq"].astype(float)
    temp=temp.round({"freq":2})
    print(temp.sort_values("freq",ascending=False).reset_index(drop=True).head(num_top_venues))
    print("\n")

----Berczy Park----
                venue  freq
0         Coffee Shop  0.07
1        Cocktail Bar  0.05
2                Café  0.04
3         Cheese Shop  0.04
4  Italian Restaurant  0.04


----Brockton / Parkdale Village / Exhibition Place----
            venue  freq
0            Café  0.13
1  Breakfast Spot  0.09
2     Coffee Shop  0.09
3             Gym  0.04
4          Bakery  0.04


----Business reply mail Processing CentrE----
           venue  freq
0    Yoga Studio  0.06
1  Garden Center  0.06
2    Pizza Place  0.06
3     Comic Shop  0.06
4     Restaurant  0.06


----CN Tower / King and Spadina / Railway Lands / Harbourfront West / Bathurst Quay / South Niagara / Island airport----
              venue  freq
0   Airport Service  0.18
1    Airport Lounge  0.12
2  Airport Terminal  0.12
3           Airport  0.06
4               Bar  0.06


----Central Bay Street----
                venue  freq
0         Coffee Shop  0.20
1                Café  0.06
2  Italian Restaurant  0.06
3    

Function to sort the venues in descending order

In [148]:
def return_most_common_venues(row,num_top_venues):
    row_categories=row.iloc[1:]
    row_categories_sorted=row_categories.sort_values(ascending=False)
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood

In [149]:
num_top_venues=10
indicators=["st","nd","rd"]
columns=["Neighborhood"]
for ind in np.arange(num_top_venues):
    try:
        columns.append("{}{} Most Common Venue".format(ind+1,indicators[ind]))
    except:
        columns.append("{}th Most Common Venue".format(ind+1))
neighborhoods_venuess_sorted=pd.DataFrame(columns=columns)
neighborhoods_venuess_sorted["Neighborhood"]=toronto_grouped["Neighborhood"]
for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venuess_sorted.iloc[ind,1:]=return_most_common_venues(toronto_grouped.iloc[ind,:],num_top_venues)
neighborhoods_venuess_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Seafood Restaurant,Café,Bakery,Italian Restaurant,Cheese Shop,Farmers Market,Restaurant,Beer Bar
1,Brockton / Parkdale Village / Exhibition Place,Café,Breakfast Spot,Coffee Shop,Yoga Studio,Gym,Pet Store,Performing Arts Venue,Nightclub,Italian Restaurant,Intersection
2,Business reply mail Processing CentrE,Yoga Studio,Spa,Gym / Fitness Center,Garden Center,Fast Food Restaurant,Farmers Market,Light Rail Station,Comic Shop,Pizza Place,Butcher
3,CN Tower / King and Spadina / Railway Lands / ...,Airport Service,Airport Lounge,Airport Terminal,Airport,Bar,Coffee Shop,Plane,Sculpture Garden,Boutique,Boat or Ferry
4,Central Bay Street,Coffee Shop,Italian Restaurant,Café,Sandwich Place,Thai Restaurant,Japanese Restaurant,Middle Eastern Restaurant,Salad Place,Ice Cream Shop,Burger Joint


Cluster Neighborhoods: Run k-means to cluster the neighborhood into 5 clusters

In [163]:
kclusters=11
toronto_grouped_clustering=toronto_grouped.drop("Neighborhood",1)
kmeans=KMeans(n_clusters=kclusters,random_state=0).fit(toronto_grouped_clustering)
kmeans.labels_[0:10]

array([3, 3, 6, 9, 3, 7, 6, 3, 3, 5], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood

In [177]:
neighborhoods_venuess_sorted.insert(0,"Cluster Labels",kmeans.labels_)
toronto_merged=dfnew1
toronto_merged=toronto_merged.join(neighborhoods_venuess_sorted.set_index("Neighborhood"),on="Neighborhood")
toronto_merged.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.67635739999999,-79.2930312,10,Coffee Shop,Pub,Health Food Store,Trail,Yoga Studio,Dog Run,Dessert Shop,Diner,Discount Store,Distribution Center
1,M4K,East Toronto,The Danforth West / Riverdale,43.6795571,-79.352188,3,Greek Restaurant,Coffee Shop,Italian Restaurant,Furniture / Home Store,Bookstore,Ice Cream Shop,Yoga Studio,Pub,Pizza Place,Lounge
2,M4L,East Toronto,India Bazaar / The Beaches West,43.6689985,-79.31557159999998,6,Sandwich Place,Fast Food Restaurant,Food & Drink Shop,Brewery,Liquor Store,Restaurant,Burrito Place,Italian Restaurant,Fish & Chips Shop,Pub
3,M4M,East Toronto,Studio District,43.6595255,-79.340923,3,Café,Coffee Shop,Bakery,Brewery,Gastropub,American Restaurant,Diner,Bar,Ice Cream Shop,Fish Market
4,M4N,Central Toronto,Lawrence Park,43.7280205,-79.3887901,1,Park,Bus Line,Swim School,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant


Finally, let's visualize the resulting clusters

In [181]:
map_clusters=folium.Map(location=[latitude,longitude],zoom_start=11)
x=np.arange(kclusters)
ys=[i+x+(i*x)**2 for i in range(kclusters)]
colors_array=cm.rainbow(np.linspace(0,1,len(ys)))
rainbow=[colors.rgb2hex(i) for i in colors_array]
markers_colors=[]
for lat,lng,poi,cluster in zip(toronto_merged["Latitude"],toronto_merged["Longitude"],toronto_merged["Neighborhood"],toronto_merged["Cluster Labels"]):
    label=folium.Popup(str(poi)+" Cluster "+str(cluster),parse_html=True)
    folium.CircleMarker([lat,lng],radius=5,popup=label,color=rainbow[cluster-1],fill=True,fill_color=rainbow[cluster-1],fill_opacity=0.7).add_to(map_clusters)
map_clusters

Examining clusters: Cluster 1

In [182]:
toronto_merged.loc[toronto_merged["Cluster Labels"]==0,toronto_merged.columns[[1]+list(range(5,toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
25,Downtown Toronto,0,Café,Japanese Restaurant,Restaurant,Bar,Italian Restaurant,Bookstore,Bakery,Beer Bar,Beer Store,Sandwich Place
26,Downtown Toronto,0,Café,Coffee Shop,Mexican Restaurant,Dessert Shop,Vietnamese Restaurant,Bakery,Bar,Grocery Store,Vegetarian / Vegan Restaurant,Gaming Cafe
31,West Toronto,0,Pharmacy,Bakery,Grocery Store,Athletics & Sports,Gym / Fitness Center,Middle Eastern Restaurant,Music Venue,Pizza Place,Café,Brewery
34,West Toronto,0,Mexican Restaurant,Café,Thai Restaurant,Grocery Store,Speakeasy,Bookstore,Cajun / Creole Restaurant,Music Venue,Diner,Bakery


Similarly by changing the cluster label to any other value we can find the other 10 clusters and what to they compose of.